In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import os
import sys
sys.path.append(os.path.abspath("../Notebooks"))

from helper_functions import  *


In [15]:

# Read compressed csv with built-in gzip support
data_reviews = decompress_pickle('../Data/reviews_zipped.pbz2') 

df_recipes= pd.read_csv('../Data/final_datav3.csv', compression='gzip') 

df_recipes['ingredient_ids_str'] = df_recipes.loc[:, 'ingredient_ids']
df_recipes['ingredient_ids']=df_recipes.ingredient_ids.apply(lambda x : convert_type(x))



Create a graph

In [16]:
#!pip install networkx
import networkx as nx

G = nx.MultiDiGraph()
G.add_nodes_from(df_recipes['id'].values, name=df_recipes['name'].values, n_ingredients=df_recipes['n_ingredients'].values)
print(G)

'''
col_one_list = df_marvel['edges'].tolist()
col_one_listDC=(df_DC['edges'].tolist())

edges_values_list= col_one_list + col_one_listDC

def flatten(l):
    return [item for sublist in l for item in sublist]

edges_values_tuples = flatten(edges_values_list)

G.add_edges_from(edges_values_tuples)
print(G)
'''

MultiDiGraph with 178265 nodes and 0 edges


"\ncol_one_list = df_marvel['edges'].tolist()\ncol_one_listDC=(df_DC['edges'].tolist())\n\nedges_values_list= col_one_list + col_one_listDC\n\ndef flatten(l):\n    return [item for sublist in l for item in sublist]\n\nedges_values_tuples = flatten(edges_values_list)\n\nG.add_edges_from(edges_values_tuples)\nprint(G)\n"

In [13]:
for index,row in df_recipes.iterrows():
    for item in row['ingredient_ids']:
        print (item)
        df_filtr=df_recipes[df_recipes['ingredient_ids_str'].str.contains(str(item)) ]
        edges=[(row['id'], id, {'ingredient: item'}) for id in df_filtr['id'].values if id!=row['id']]
    break
print(edges)

7933
4694
4795
3723
840
5006
6270
[(137739, 59389, {'ingredient: item'}), (137739, 75452, {'ingredient: item'}), (137739, 67547, {'ingredient: item'}), (137739, 107517, {'ingredient: item'}), (137739, 63986, {'ingredient: item'}), (137739, 76808, {'ingredient: item'}), (137739, 24701, {'ingredient: item'}), (137739, 49262, {'ingredient: item'}), (137739, 54272, {'ingredient: item'}), (137739, 47366, {'ingredient: item'}), (137739, 27087, {'ingredient: item'}), (137739, 59952, {'ingredient: item'}), (137739, 54100, {'ingredient: item'}), (137739, 25775, {'ingredient: item'}), (137739, 33165, {'ingredient: item'}), (137739, 93958, {'ingredient: item'}), (137739, 26995, {'ingredient: item'}), (137739, 53402, {'ingredient: item'}), (137739, 59632, {'ingredient: item'}), (137739, 35653, {'ingredient: item'}), (137739, 44045, {'ingredient: item'}), (137739, 81185, {'ingredient: item'}), (137739, 26835, {'ingredient: item'}), (137739, 42570, {'ingredient: item'}), (137739, 108804, {'ingredien

In [7]:
name = 'test'
df_filtr=df_recipes[df_recipes.loc[lambda x : x ]]
#df_filtr=df_recipes[df_recipes['ingredient_ids'].str.contains('4694')]
edges=[(name, id, {'ingredient: idx'}) for id in df_filtr['id'].values]
edges

KeyError: "None of [Float64Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              ...\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n             dtype='float64', length=178265)] are in the [columns]"

In [20]:
for id in df_filtr['id'].values:
    print(id)

137739
37595
340923
345720
358881
26117
291912
253293
50031
449467
305472
192840
147225
224068
9022
435039
191775
349828
12790
31175
114352
348017
